## This file helps to Build and test the LLama model and Langchain functions

In [1]:
# Importing packages

from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

In [2]:
#setting up env
load_dotenv()

True

In [3]:
#Building LLM
llm = ChatGroq(
    model='llama3-8b-8192',
    groq_api_key=os.getenv("GROQ_API_KEY"),
    temperature=0.5
    )

In [7]:
#Getting response 
response = llm.invoke("Ask me a question to determine my career path along with 4 options to chose from.")
print(response)

content="Here's a question to help determine your career path:\n\nWhen working on a project, what motivates you most?\n\nA) The opportunity to be creative and come up with innovative solutions\nB) The chance to work with people and build strong relationships\nC) The challenge of solving complex problems and overcoming obstacles\nD) The sense of accomplishment and achieving specific goals\n\nWhich option resonates with you the most?" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 28, 'total_tokens': 111, 'completion_time': 0.069166667, 'prompt_time': 0.003874421, 'queue_time': 0.020889679, 'total_time': 0.073041088}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None} id='run-185f54cd-cdbc-4fc8-8996-b8c60b8ba80a-0' usage_metadata={'input_tokens': 28, 'output_tokens': 83, 'total_tokens': 111}


In [8]:
from langchain.document_loaders import WebBaseLoader

url = "https://www.google.com/"
loader = WebBaseLoader(url)
docs = loader.load()

print(docs[0].page_content, end = ' ')  # Extracted text from the webpage

GoogleSearch Images Maps Play YouTube News Gmail Drive More »Web History | Settings | Sign in Advanced searchAdvertisingBusiness SolutionsAbout Google© 2025 - Privacy - Terms  

In [ ]:
import chromadb
import pandas as pd
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# Load API keys
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Load O*NET data (assuming CSV format)
df = pd.read_csv("onet_careers.csv")  # Ensure this file has career details

# Extract relevant fields
career_docs = [
    f"Career: {row['Title']}. Description: {row['Description']}. Skills: {row['Skills']}. Education: {row['Education']}."
    for _, row in df.iterrows()
]

# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
career_collection = chroma_client.get_or_create_collection("onet_careers")

# Store career documents as vectors
vectorstore_careers = Chroma.from_texts(
    career_docs, OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY), persist_directory="./chroma_db"
)
vectorstore_careers.persist()

print("✅ O*NET Career Data stored in ChromaDB!")

In [ ]:
def get_best_career(user_answers):
    """Retrieve the most relevant careers based on user responses."""
    user_profile = " ".join(user_answers)  # Combine answers into a profile
    results = vectorstore_careers.similarity_search(user_profile, k=3)  # Get top 3 matching careers
    return [doc.page_content for doc in results]

In [ ]:
def generate_career_recommendation(user_answers):
    """Uses RAG to retrieve career matches & LLaMA to generate recommendations."""
    careers = get_best_career(user_answers)  # Fetch relevant careers

    prompt = f"""
    Based on the user's responses:
    {user_answers}
    
    Here are the most relevant careers:
    {', '.join(careers)}

    Provide a final career recommendation with an explanation.
    """
    
    response = llm.invoke(prompt)
    return response.content